In [3]:
import json

## Activity net dataset video file counts 

In [7]:
#total files
import os

def count_files(folder_path):
    file_count = 0
    for root, dirs, files in os.walk(folder_path):
        file_count += len(files)
    return file_count

folder_path = 'E:\Datasets\ActivityNet\data\ActivityNet_200'  # Replace with the actual folder path
total_files = count_files(folder_path)
print("Total files:", total_files)

Total files: 14819


In [13]:
folder_path = 'E:\Datasets\ActivityNet\data\ActivityNet_200\\testing'  # Replace with the actual folder path
total_files = count_files(folder_path)
print("testing files:", total_files)

testing files: 3703


In [14]:
folder_path = 'E:\Datasets\ActivityNet\data\ActivityNet_200\\training'  # Replace with the actual folder path
total_files = count_files(folder_path)
print("training files:", total_files)

training files: 7453


In [15]:
folder_path = 'E:\Datasets\ActivityNet\data\ActivityNet_200\\validation'  # Replace with the actual folder path
total_files = count_files(folder_path)
print("validation files:", total_files)

validation files: 3663


## Activity net captions explore


In [3]:
import pickle
a_n_captions_file_path = "E:\\Datasets\\ActivityNet\\activity_net_captions.pickle"
with open(a_n_captions_file_path,"rb") as f:
    captions = pickle.load(f)


In [5]:
for key in captions.keys():
    print(key)
    print(captions[key])
    break

QOlSCBRmfWY
{'start': array([ 0.83, 17.37, 56.26]), 'end': array([19.86, 60.81, 79.42]), 'text': array(['A young woman is seen standing in a room and leads into her dancing.',
       ' The girl dances around the room while the camera captures her movements.',
       ' She continues dancing around the room and ends by laying on the floor.'],
      dtype=object), 'transcript': array(['none', 'none', 'none'], dtype=object)}


In [6]:
captions['QOlSCBRmfWY']

{'start': array([ 0.83, 17.37, 56.26]),
 'end': array([19.86, 60.81, 79.42]),
 'text': array(['A young woman is seen standing in a room and leads into her dancing.',
        ' The girl dances around the room while the camera captures her movements.',
        ' She continues dancing around the room and ends by laying on the floor.'],
       dtype=object),
 'transcript': array(['none', 'none', 'none'], dtype=object)}

## find least frequent 50 labels from train+validation set

In [13]:
import json

# Specify the path to your JSON file
label_video_ct_path = "E:\\Datasets\\ActivityNet\\data\\label_video_ct.json"

# Open the file and load its contents
with open(label_video_ct_path, "r") as file:
    label_video_ct = json.load(file)


least_50_label_video_ct = label_video_ct[-50:]

# Write the list to the JSON file
with open("data\\least_50_label_video_ct.json", "w") as file:
    json.dump(least_50_label_video_ct, file,indent=4)

## make {label: [video_id,..]} and store to generate video path, start, end file for caption generator

In [68]:
least_50_label_video_ct_path = "E:\\Datasets\\ActivityNet\\data\\least_50_label_video_ct.json"

with open(least_50_label_video_ct_path,'r') as file: 
    least_50_label_video_ct = json.load(file)

least_50_label_list = [item[0] for item in least_50_label_video_ct]


In [19]:
with open("E:\\Datasets\\ActivityNet\\activity_net.v1-3.min.json",'rb') as file:
    data = json.load(file)

# take only video informations from database object
videos = data['database']

In [54]:
label_to_video_id = {}
c=0
for key in videos.keys():
    # print(videos[key])
    video = videos[key]
    if video['subset'] == 'training' or video['subset']=='validation':
    
        if video['annotations'][0]['label'] in least_50_label_list: # choose the first annotation label as the video label  
            # print("yes")
            c+=1
            # else: print("no")
print(c)
    

2884


In [56]:
## checking whether least 50 label (train+valid) video counts == count c
sum([item[1] for item in least_50_label_video_ct])

2884

In [59]:
# selecting videos from only training set that are among least_50_label_list
# there are 1919 videos 
label_to_video_id = {}
c=0
for key in videos.keys():
    # print(videos[key])
    video = videos[key]
    if video['subset'] == 'training':
    
        if video['annotations'][0]['label'] in least_50_label_list: # choose the first annotation label as the video label  
            label = video['annotations'][0]['label']
            c+=1
            if label not in label_to_video_id: 
                label_to_video_id[label]=[key]
            else: 
                label_to_video_id[label].append(key)
            # else: print("no")
print(c)

1919


In [66]:
#checking label and video counts to match with counter c
print("label count:", len(label_to_video_id.keys()))
print("video count:", sum([len(label_to_video_id[key]) for key in label_to_video_id.keys()]))

label count: 50
video count: 1919


In [67]:
# save the file for generating video paths and later concatenating start,end segments from  Activity net captions
with open("label_to_video_ids_least_50_label_trainset.json",'w') as file:
    json.dump(label_to_video_id,file, indent=4)

## generate least_50_label_video_id_segment_caps

    

In [ ]:
# format: 
# {
#     "label": [
#         {
#             video_id: [
#                 {
#                     #these are segments
#                     "caption": "A furniture renovation company advertises their services by showing several homes and tools they have worked on and use and what type of work they do.",
#                     "start": 0.0,
#                     "end": 50.48,
#                     "path": "Polishing_forniture/Ou24uqaFRPg.mp4"
#                 },
#                 {
#                     "caption": " A man in an apron carefully puts a colored substance on a plate.",
#                     "start": 9.84,
#                     "end": 12.87,
#                     "path": "Polishing_forniture/Ou24uqaFRPg.mp4"
#                 }],
#             video_id: [
#                 {
#                     ...
#                 }]
#         }],

# }

In [12]:
# load activity net captions 
import pickle
a_n_captions_file_path = "E:\\Datasets\\ActivityNet\\activity_net_captions.pickle"
with open(a_n_captions_file_path,"rb") as f:
    captions = pickle.load(f)

In [4]:
# load least 50 labels (train set), label to video ids
with open("E:\\Datasets\\ActivityNet\\data\\label_to_video_ids_least_50_label_trainset.json",'r') as file:
    label_to_ids_least_50 = json.load(file)



In [5]:
for key in captions.keys():
    print(captions[key])
    break

{'start': array([ 0.83, 17.37, 56.26]), 'end': array([19.86, 60.81, 79.42]), 'text': array(['A young woman is seen standing in a room and leads into her dancing.',
       ' The girl dances around the room while the camera captures her movements.',
       ' She continues dancing around the room and ends by laying on the floor.'],
      dtype=object), 'transcript': array(['none', 'none', 'none'], dtype=object)}


In [82]:
for key in captions.keys():
    video = captions[key]
    print(key)
    for i in range(len(video['start'])):
        print("i",i)
        start = video['start'][i]
        end = video['end'][i]
        text = video['text'][i]
        print(f'''
        video_id: {key}
        caption: {text}
        start: {start} 
        end: {end}
        ''')
    break

QOlSCBRmfWY
i 0

        video_id: QOlSCBRmfWY
        caption: A young woman is seen standing in a room and leads into her dancing.
        start: 0.83 
        end: 19.86
        
i 1

        video_id: QOlSCBRmfWY
        caption:  The girl dances around the room while the camera captures her movements.
        start: 17.37 
        end: 60.81
        
i 2

        video_id: QOlSCBRmfWY
        caption:  She continues dancing around the room and ends by laying on the floor.
        start: 56.26 
        end: 79.42
        


In [24]:
least_50_label_video_id_segment_caps={}
for label,video_ids in label_to_ids_least_50.items():
    # print("\t",label)
    video_id_to_segment_caps = {}
    for id in video_ids: 
        
        try: 
            cap_data = captions[id] #captions is loaded from activity_net_captions.pickle
            # if id == "adz3kDJLCF4":

            # # print(id)
            #     print(cap_data)
            segment_cap_list = []
            for i in range(len(cap_data['start'])):
                segment_cap ={}

                text = cap_data['text'][i].strip()
                start = cap_data['start'][i]
                end = cap_data['end'][i]
                path = label.replace(" ","_")+"/"+id+".mp4"
                
                segment_cap['caption'] = text
                segment_cap['start'] = float(start) # converting to float, beacuse id "adz3kDJLCF4" has start array of integers
                segment_cap['end'] = float(end) 
                segment_cap['path'] = path

                segment_cap_list.append(segment_cap)
                # print(f'''
                # video_id: {id}
                # caption: {text}
                # start: {start} 
                # end: {end}
                # path: {label.replace(" ","_")}/{id}.mp4
                # ''')
        except: 
            print("captions not found for video id: ",id, "\nlabel: ",label)
            
            # segment_cap_list=[{"caption":"-",
            #                    "start":"0.0",
            #                    "end":"0.0",
            #                    "path":"-"
            #                    }]
            
            
        video_id_to_segment_caps[id] = segment_cap_list
        
    least_50_label_video_id_segment_caps[label] = video_id_to_segment_caps
    
# print(least_50_label_video_id_segment_caps)

with open("data/least_50_label_video_id_segment_caps.json","w") as f: 
    json.dump(least_50_label_video_id_segment_caps,f,indent=4)
   

captions not found for video id:  JkxO6IhuhOs 
label:  Bungee jumping


In [54]:
# check the length of total videos of random label : 'Carving jack-o-lanterns', in label_to_ids_least_50 
len(label_to_ids_least_50['Carving jack-o-lanterns'])

36

In [55]:
#check the length of total videos of random label: 'Carving jack-o-lanterns' on least least_50_label_video_id_segment_caps
len(least_50_label_video_id_segment_caps['Carving jack-o-lanterns'])

36